**This kernel was copied from [MinMax.Goose Go kernel](http://https://www.kaggle.com/egrehbbt/minmax-goose-go) so please give it upvoit.**

**In the previous kernel, the agent used to play against one opponent, now it plays against more than one opponent**
**The only problem here is that the minimax algorithm takes time to get the action out**

# Action class

In [ ]:
from collections import defaultdict
import random
import numpy as np
from enum import Enum
from typing import List,Tuple,Optional
import multiprocessing


In [ ]:
class Action(Enum):
    NORTH = "NORTH"
    SOUTH = "SOUTH"
    EAST  = "EAST"
    WEST  = "WEST"
    def __str__(self):
        return self.value
    def __repr__(self):
        return self.value
    
    def opposite(self):
        return _ACTION_TO_OPPOSITE[self]

    def to_tuple(self):
        return _ACTION_TO_TUPLE[self]
    
    @classmethod
    def from_tuple(cls,t:Tuple[int,int]):
        if t[0] == 0:
            if t[1] == 1:
                return cls.NORTH
            if t[1] == -1:
                return cls.SOUTH
        if t[1] == 0:
            if t[0] == 1:
                return cls.EAST
            if t[0] == -1:
                return cls.WEST
        raise ValueError(f"Can't convert tuple {t}")
        
        
_ACTION_TO_OPPOSITE = {
    Action.NORTH:Action.SOUTH,
    Action.SOUTH:Action.NORTH,
    Action.EAST:Action.WEST,
    Action.WEST:Action.EAST
}

_ACTION_TO_TUPLE = {
    Action.NORTH:(0,1),
    Action.SOUTH:(0,-1),
    Action.EAST:(1,0),
    Action.WEST:(-1,0)
}

# Board class

In [ ]:
NUM_ROWS = 7
NUM_COLUMNS = 11
HUNGER_RATE = 40
_PREVIOUS_STATES = []


In [ ]:
class Board:
    def __init__(self):
        self._ps = np.zeros((NUM_COLUMNS,NUM_ROWS),dtype=Point)
        for x in range(NUM_COLUMNS):
            for y in range(NUM_ROWS):
                self._ps[x,y] = Point(x,y,board=self)
        self._food = []
    def __repr__(self):
            return f"Board {NUM_ROWS}X{NUM_COLUMNS}"
    def __getitem__(self,item):
            x,y = item
            return self._ps[x % NUM_COLUMNS,y % NUM_ROWS]
    def add_food(self,x: int,y:int):
            p = self[x, y]
            p.food = True
            self._food.append(p)
    def food_positions(self):
            return self._food
            

# Point class

In [ ]:
class Point:
    def __init__(self,x:int,y:int,board:Board):
        self.x = x
        self.y = y
        self.board = board
        
        self.goose: Optional[Goose] = None
        self.food: bool = False
    
    def __repr__(self):
        return f"[{self.x},{self.y}]"
    def __hash__(self):
        return self.y * NUM_COLUMNS + self.x
    def __sub__(self,other:"Point"):
        return self.board[self.x - other.x,self.y - other.y]
    def __add__(self,other:"Point"):
        return self.board[self.x + other.x,self.y + other.y]
    def apply(self,action:"Action"):
        return _transform(self,action)
    def to_tuple(self):
        return self.x,self.y
    def to_flat(self):
        return (NUM_ROWS - self.y - 1) * NUM_COLUMNS + self.x
    def distance_from(self,other:"Point"):
        _, d = self.dirs_to(other)
        return d
    def dirs_to(self,other:"Point"):
        return _dirs_to(dx=self.x - other.x,dy = self.y - other.y)
    def nearby_points(self,radius:int = 1):
        return _nearby_points(self,radius)
    def food_distance(self):
        food_positions = self.board.food_positions()
        if not food_positions:
            return np.inf
        return min(self.distance_from(x) for x in food_positions)


In [ ]:
from functools import lru_cache
import itertools
@lru_cache(256)
def _nearby_points(p:Point,radius: int = 1):
    if radius <= 0:
        return []
    points = []
    for actions in itertools.product(Action,repeat=radius):
        _p = p
        for a in actions:
            _p = _p.apply(a)
        if (_p.x,_p.y) != (p.x,p.y) and _p not in points:
            points.append(_p)
    return points
                        
@lru_cache(128)
def _dirs_to(dx:int,dy:int):
    if dx == 0 and dy == 0:
        return [],0
    if abs(dx) > NUM_COLUMNS / 2:
        dx -= np.sign(dx) * NUM_COLUMNS
    if abs(dy) > NUM_ROWS / 2:
        dy -= np.sign(dy) * NUM_ROWS
    
    ret = []
    if dx > 0:
        ret.append(Action.WEST)
    elif dx < 0:
        ret.append(Action.EAST)
    if dy > 0:
        ret.append(Action.SOUTH)
    elif dy < 0:
        ret.append(Action.NORTH)
    return ret,abs(dx) + abs(dy)

def _transform(p:Point,a:Action):
    x ,y = a.to_tuple()
    return p.board[p.x + x,p.y + y]

def is_starving_step(step: int):
    return (step + 1) % HUNGER_RATE == 0

def find_opponent_allowed_actions(goose:"Goose",opponents:List["Goose"],starving:bool = False):
    allowed_actions = goose.self_collision_free_actions
    if not allowed_actions:
        return allowed_actions
    forbidden_actions = []
    for op in opponents:
        if not op.is_active or not op.self_collision_free_actions:
            continue
        
        op_points = op.positions[:-1]
        if starving:
            op_points = op_points[:-1]
        
        for a in allowed_actions:
            if a not in forbidden_actions:
                next_head_position = goose.head.apply(a)
                if next_head_position in op_points:
                    forbidden_actions.append(a)
    allowed_actions = [x for x in allowed_actions if x not in forbidden_actions]
    return allowed_actions

# Goose class

In [ ]:
class Goose:
    def __init__(
        self,
        id: int,
        step: int,
        positions:List[Point],
        last_action: Optional[Action],
        reward: Optional[int] = None):
        self.id = id
        self.step = step
        self.positions = positions
        self.last_action = last_action
        
        self.reward = (
            reward if reward is not None else (self.step + 1) * 100 + self.length
        )
        self.is_active = bool(positions)
        if self.is_active:
            self.head = positions[0]
            self.self_opposite_free_actions = self.__get_self_opposite_free_actions()
            self.self_collision_free_actions = self.__get_self_collision_free_actions()
     ########       
        else:
            self.head = None
            self.self_opposite_free_actions = []
            self.self_collision_free_actions = [] 
            
    ########
    def __get_self_opposite_free_actions(self):
        if not self.last_action:
            return list(Action)
        not_allowed_action = self.last_action.opposite()
        return [a for a in Action if a != not_allowed_action]
    
    def __get_self_collision_free_actions(self):
        new_body_positions = self.positions[:-1]
        allowed_actions = []
        for a in self.self_opposite_free_actions:
            head_point = self.head.apply(a)
            for n,body_point in  enumerate(new_body_positions):
                if ([head_point.x,head_point.y] == [body_point.x,body_point.y]):
                    break
                elif(n+1==len(new_body_positions)):
                    allowed_actions.append(a)
                    break
        return allowed_actions
    def __repr__(self):
        return f"Goose(id={self.id},positions={self.positions})"
    def __iter__(self):
        return iter(self.positions)    
    def __len__(self):
        return self.length
    def __bool__(self):
        return self.is_active
    def to_flat(self):
        return [x.to_flat() for x in self.positions]
    
    @property
    def tail(self):
        return self.positions[-1]
    def distance_from(self,point:"Point"):
        _, d = self.dirs_to(point) 
    def dirs_to(self,point:"Point"):
        p = self.head
        return _dirs_to(dx=p.x - point.x,dy=p.y - point.y)
    
    @property    
    def length(self):
        return len(self.positions)
    @property
    def length(self):
        return len(self.positions)

    def can_move(self):
        return bool(self.self_collision_free_actions)

    def __get_self_opposite_free_actions(self):
        if not self.last_action:
            return list(Action)

        not_allowed_action = self.last_action.opposite()
        return [a for a in Action if a != not_allowed_action]

    def __get_self_collision_free_actions(self):
        new_body_positions = self.positions[:-1]
        allowed_actions = []
        for a in self.self_opposite_free_actions:
            head_point = self.head.apply(a)
            for n,body_point in  enumerate(new_body_positions):
                if ([head_point.x,head_point.y] == [body_point.x,body_point.y]):
                    break
                elif(n+1==len(new_body_positions)):
                    allowed_actions.append(a)
                    break
        return allowed_actions

    def kick(self, last_action: Optional[Action] = None):
        return self.__class__(
            id=self.id,
            step=self.step,
            positions=[],
            last_action=last_action,
            reward=self.reward,
        )

    def apply(self, action: Action, food: List["Point"]):
        assert self.is_active

        if self.last_action and action == self.last_action.opposite():
            return self.kick(action)

        head_position = self.head.apply(action)

        body_positions = self.positions
        if head_position not in food:
            body_positions = body_positions[:-1]

        if head_position in body_positions:
            return self.kick(action)

        new_goose = self.__class__(
            id=self.id,
            step=self.step + 1,
            positions=[head_position] + body_positions,
            last_action=action,
        )

        if is_starving_step(self.step):
            new_goose.positions = new_goose.positions[:-1]
            if not new_goose.positions:
                return self.kick(action)

        return new_goose


# Node class

In [ ]:
class _Node:
    def __init__(
       self,
       g1:Goose,
       op_geese:List[Goose],
       food:List[Point],
       step:int,
       remaining_steps:int,
       max_sub_nodes:float=500
    ):
        self.g1 = g1
        self.op_geese = op_geese
        self.food = food
        self.step = step
        self.remaining_steps = remaining_steps
        self.max_sub_nodes = max_sub_nodes
        self.sub_nodes = [] # [(node,score),(node,score),...]
        self.action_to_score = {}
        self.score = self._calculate_score()
        
    def __repr__(self):
        return f"Node step={self.step}"
    
    def best_actions(self):
        if not self.action_to_score:
            return []
        max_score = max(self.action_to_score.values())
        return [a for a, s in self.action_to_score.items() if s == max_score]

    @staticmethod
    def _reward_score(my,ops:List[Goose]):
        my_reward = []
        for op in ops:
            if my.reward >= op.reward:
                my_reward.append([1])
            else:
                my_reward.append([0])
        return np.sum(my_reward)/len(my_reward)

    @staticmethod
    def __estimate_space_score(g1:Goose,ops:List[Goose]):
        head = g1.head
        if g1.length > 1:
            exp = [g1.positions[1]]
        elif g1.last_action:
            exp = [g1.head.apply(g1.last_action.opposite())]
        else:
            exp = []
        occupied_points =  g1.positions
        
        for op in ops:
            occupied_points += op.positions
                
        space ,total = 0,0
        for x in head.nearby_points(radius=1):
            if x not in exp:
                total += 1
                if x not in occupied_points:
                    space += 1
        return 0.5 * (space + 1)/(total + 1)
            
    def _calculate_score(self):

        my = self.g1
        ops = self.op_geese
        if self.remaining_steps <= 0:
            return self._reward_score(my,ops)
        
        starving = is_starving_step(self.step)
        my_actions = find_opponent_allowed_actions(my,ops,starving=starving)
        ops_actions = []
        for op in ops:
            ops_actions.append(find_opponent_allowed_actions(op,[ x for x in ops if x.id != op.id] + [my] ,starving=starving)) 
        if not ops_actions:
            if my_actions:
                return 1
            
        my_reward = []
            
        if not my_actions:
            for n,op_actions in enumerate(ops_actions):
                if not op_actions:
                    if self._reward_score(my,[ops[n]]):
                        my_reward.append(1)
                    else:
                        my_reward.append(0)  
                else:
                    my_reward.append(0)                     
                return np.sum(my_reward)/len(my_reward)
            
            
        num_sub_nodes = 1
        for actions in  ops_actions:
            if actions != []:
                num_sub_nodes = num_sub_nodes * len(actions)
                
        num_nodes_per_sub_node = self.max_sub_nodes / num_sub_nodes
        if num_nodes_per_sub_node < 1:
            return self.__estimate_space_score(my,ops)
        
        food = self.food
        my_action_to_scores = defaultdict(list)
        ops_actions_after_modification = []
        for op_actions in ops_actions:
            if op_actions == []:
                ops_actions_after_modification.append(['no_actions'])
            else:
                ops_actions_after_modification.append(op_actions)
        
        for a in itertools.product(my_actions,*ops_actions_after_modification):
            next_my = my.apply(a[0],food)
            next_heads = []
            next_heads.append(next_my.head)
            next_ops = []
            for n,op in enumerate(ops):
                if a[n+1] != 'no_actions':
                    op.apply(a[n+1],food)
                else:
                    op.apply(a[0],food)
                next_ops.append(op)
                
            for next_op in next_ops:
                if next_op.is_active:
                    next_heads.append(next_op.head)
            next_food = [f for f in food if f not in next_heads]
            
            next_node = _Node(
                g1=next_my,
                op_geese = next_ops,
                food=next_food,
                step=self.step + 1,
                remaining_steps=self.remaining_steps - 1,
                max_sub_nodes=num_nodes_per_sub_node,
            )
            
            score = next_node.score
            self.sub_nodes.append((next_node, score))
            my_action_to_scores[a[0]].append(score)
         
        for action, scores in my_action_to_scores.items():
                self.action_to_score[action] = min(scores)
        del next_node
        return max(self.action_to_score.values())

# State class

In [ ]:

def find_last_action(goose_id: int, current_positions: List[Point]):
    if not _PREVIOUS_STATES or not current_positions:
        return

    head = current_positions[0]
    previous_state = _PREVIOUS_STATES[-1]
    previous_goose = previous_state.geese[goose_id]
    previous_head = previous_goose.head
    if not previous_head:
        return

    a, d = previous_head.dirs_to(head)
    assert d == 1
    return a[0]


In [ ]:
class State:
    def __init__(self,obs,conf):
        self.step = obs['step']
        self.my_id = obs['index']
        self.remaining_overage_time = obs["remainingOverageTime"]
        self.remaining_steps = conf["episodeSteps"] - self.step - 1  
        
        assert conf["columns"] == NUM_COLUMNS
        assert conf["rows"] == NUM_ROWS
        assert conf["hunger_rate"] == HUNGER_RATE
        
        self.board = Board()
        def __flat_to_point(_x:int):
            assert 0 <= _x < NUM_COLUMNS * NUM_ROWS
            return _x % NUM_COLUMNS, NUM_ROWS - _x // NUM_COLUMNS - 1
        
        # FOOD
        for p in obs["food"]:
            x, y = __flat_to_point(p)
            self.board.add_food(x, y)
        self.food = self.board.food_positions()
        
        # GEESE
        self.geese = {}
        for i, pp in enumerate(obs["geese"]):
            goose_positions = []
            for p in pp:
                x, y = __flat_to_point(p)
                point = self.board[x, y]
                goose_positions.append(point)
            last_action = find_last_action(
                goose_id=i, current_positions=goose_positions
            )
            goose = Goose(
                id=i, positions=goose_positions, step=self.step, last_action=last_action
            )
            self.geese[i] = goose

        self.my_goose = self.geese[self.my_id]
        self.opponents = [
            x for x in self.geese.values() if x.id != self.my_id and x.is_active
        ]               
    def __repr__(self):
        return f"State(step={self.step})"

    def next_action(self):
        self_allowed_actions = self.my_goose.self_collision_free_actions
        if not self_allowed_actions:
            return random.choice(list(Action))
        elif len(self_allowed_actions) == 1:
            return self_allowed_actions[0]    

        opponent_allowed_actions = find_opponent_allowed_actions(
            self.my_goose, self.opponents, starving=is_starving_step(self.step)
        )
        
        if not opponent_allowed_actions:
            return random.choice(list(self_allowed_actions))
        elif len(opponent_allowed_actions) == 1:
            return opponent_allowed_actions[0]
        
        node = _Node(
            self.my_goose,
            self.opponents,
            self.food,
            step=self.step,
            remaining_steps=self.remaining_steps,
        )     
        actions = node.best_actions()
        if not actions:
            return random.choice(list(opponent_allowed_actions))

        action = sorted(
            actions, key=lambda a: self.my_goose.head.apply(a).food_distance()
        )[0]
        del node
        return action
    
def create_state(obs, conf):
    global _PREVIOUS_STATES

    if _PREVIOUS_STATES and obs["step"] != _PREVIOUS_STATES[-1].step + 1:
        _PREVIOUS_STATES = []

    state = State(obs, conf)

    _PREVIOUS_STATES.append(state)
    return state


def min_max_agent(obs, conf):
    return str(create_state(obs, conf).next_action())

        
        

# Agent & Game class

In [ ]:
class Agent:
    def __init__(self,engine,positions):
        self.engine = engine
        self.positions = positions
    @property
    def name(self):
        if isinstance(self.engine,str):
            return self.engine
        elif callable(self.engine):
            print('callable')
            return self.engine.__module__+'-'+self.engine.__name__
        else:
            return "Unknown"
        

In [ ]:
class Game:
    def __init__(
        self,
        geese,
        food,
        episode_steps = 200,
        act_timeout = 1000,
        run_timeout = 120000,
        columns = 11,
        rows = 7,
        hunger_rate = 40,
        max_length = 99,
        debug = True,
    ):    
        self.geese = geese
        self.food = food
        self.episode_steps = episode_steps
        self.act_timeout = act_timeout
        self.run_timeout = run_timeout
        self.columns = columns
        self.rows = rows
        self.hunger_rate = hunger_rate
        self.max_length = max_length
        self.debug = debug

        self.__post_init__()
        
    def __post_init__(self):
        import kaggle_environments 
        self.min_food = len(self.food)
        
        env = kaggle_environments.make(
            "hungry_geese",configuration=self.config,debug=self.debug
        )
        env.reset(num_agents=len(self.geese))
        setattr(env, "reset", lambda _: None)
        geese_data = env.state[0]["observation"]["geese"]
        food_data = env.state[0]["observation"]["food"]
        
        for i, x in enumerate(self.geese):
            geese_data[i] = x.positions

        for i, x in enumerate(self.food):
            food_data[i] = x
        env.run([x.engine for x in self.geese])
        self.env = env
        
    def show(self, width=400, height=600):
        render = self.env.render(**{"mode": "ipython", "width": width, "height": height})
    @property
    def rewards(self):
        return [g["reward"] for g in self.env.steps[-1]]
    
    @property
    def config(self):
        return {
            "episodeSteps": self.episode_steps,
            "actTimeout": self.act_timeout,
            "runTimeout": self.run_timeout,
            "columns": self.columns,
            "rows": self.rows,
            "hunger_rate": self.hunger_rate,
            "min_food": self.min_food,
            "max_length": self.max_length,
        }
    

In [ ]:
TestAgent = min_max_agent
g1 = [49, 60, 61]
g2 = [12, 11, 22]
g3 = [40, 41, 42]
g4 = [50, 51, 52]

game = Game(
      geese=[
          Agent(engine=TestAgent,positions=g1),
          Agent(engine='greedy',positions=g2),
          Agent(engine='greedy',positions=g3),
          Agent(engine='greedy',positions=g3),

      ],
    food = [47,0],
    episode_steps=30,
)
game.show()

In [ ]:
"""""import kaggle_environments 
env = kaggle_environments.make("hungry_geese",debug=True)
env.run([min_max_agent,'greedy','greedy','greedy'])"""""
